# League of Legends Predictor

**Name(s)**: Jawad Najar and Ali Boussi

**Website Link**: (your website link)

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

pd.options.plotting.backend = 'plotly'
pd.set_option('display.max_columns', None)

## Step 1: Introduction

###

In [2]:
raw_data = pd.read_csv('data.csv')
raw_data.head()

/var/folders/_h/74y4pfyn6j36pp57yqqzprl80000gn/T/ipykernel_3621/3741442452.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('data.csv')


,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,participantid,side,position,playername,playerid,teamname,teamid,champion,ban1,ban2,ban3,ban4,ban5,pick1,pick2,pick3,pick4,pick5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,chemtechs,hextechs,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,void_grubs,opp_void_grubs,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,turretplates,opp_turretplates,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,gpr,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15,goldat20,xpat20,csat20,opp_goldat20,opp_xpat20,opp_csat20,golddiffat20,xpdiffat20,csdiffat20,killsat20,assistsat20,deathsat20,opp_killsat20,opp_assistsat20,opp_deathsat20,goldat25,xpat25,csat25,opp_goldat25,opp_xpat25,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,1,Blue,top,Soboro,oe:player:38e0af7278d6769d0c81d7c4b47ac1e,BRION Challengers,oe:team:733ebb9dbf22a401c0127a0c80193ca,Renekton,Karma,Caitlyn,Syndra,Thresh,Lulu,NaN,NaN,NaN,NaN,NaN,1713,0,2,3,2,9,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3152,0.9807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,15768.0,552.2942,0.278784,1072.3993,777.7933,8.0,0.2802,6.0,0.2102,5.0,26.0,0.9107,10934,7164.0,250.9282,0.253859,10275.0,NaN,NaN,231.0,220.0,11.0,NaN,NaN,8.0911,3228.0,4909.0,89.0,3176.0,4953.0,81.0,52.0,-44.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,5025.0,7560.0,135.0,4634.0,7215.0,121.0,391.0,345.0,14.0,0.0,1.0,0.0,0.0,1.0,0.0,6506.0,9853.0,172.0,6338.0,10200.0,171.0,168.0,-347.0,1.0,0.0,1.0,1.0,0.0,2.0,0.0,8462.0,11754.0,212.0,7857.0,12279.0,203.0,605.0,-525.0,9.0,0.0,1.0,1.0,0.0,2.0,0.0
1,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,2,Blue,jng,Raptor,oe:player:637ed20b1e41be1c51bd1a4cb211357,BRION Challengers,oe:team:733ebb9dbf22a401c0127a0c80193ca,Xin Zhao,Karma,Caitlyn,Syndra,Thresh,Lulu,NaN,NaN,NaN,NaN,NaN,1713,0,2,5,6,9,19,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.3152,0.9807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,11765.0,412.0841,0.208009,944.2732,650.1576,6.0,0.2102,18.0,0.6305,6.0,48.0,1.6813,9138,5368.0,188.0210,0.190220,8750.0,NaN,NaN,148.0,33.0,115.0,NaN,NaN,5.1839,3429.0,3484.0,58.0,2944.0,3052.0,63.0,485.0,432.0,-5.0,1.0,2.0,0.0,0.0,0.0,1.0,5366.0,5320.0,89.0,4825.0,5595.0,100.0,541.0,-275.0,-11.0,2.0,3.0,2.0,0.0,5.0,1.0,6854.0,7193.0,116.0,6708.0,8275.0,142.0,146.0,-1082.0,-26.0,2.0,3.0,2.0,1.0,5.0,1.0,8254.0,8958.0,135.0,7833.0,9861.0,163.0,421.0,-903.0,-28.0,2.0,4.0,2.0,1.0,5.0,1.0
2,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,3,Blue,mid,Feisty,oe:player:d1ae0e2f9f3ac1e0e0cdcb86504ca77,BRION Challengers,oe:team:733ebb9dbf22a401c0127a0c80193ca,LeBlanc,Karma,Caitlyn,Syndra,Thresh,Lulu,NaN,NaN,NaN,NaN,NaN,1713,0,2,2,3,9,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3152,0.9807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [3]:
raw_data.shape

(150588, 161)

In [4]:
raw_data['gameid'].nunique()

12549

## Step 2: Data Cleaning and Exploratory Data Analysis

#### Data Cleaning

In [5]:
df = raw_data.copy()

# filter for complete data
df = df[df['datacompleteness'] == 'complete']

# drop unnecessary columns (metadata)
meta_columns = ['participantid', 'playerid', 'teamid', 
                'datacompleteness', 'url', 'date',
                'teamname', 'playername']
df = df.drop(columns=meta_columns)

# seperate team and player stats
team_df = df[df['position'] == 'team']
player_df = df[df['position'] != 'team']

# Dropping all columnsn that are all NaN (it is a team/player stat, respectively)
player_df = player_df.dropna(axis=1, how='all')
team_df = team_df.dropna(axis=1, how='all')

# Team Adv columns 
team_df['tower_advantage'] = team_df['towers'] - team_df['opp_towers']
team_df['inhibitor_advantage'] = team_df['inhibitors'] - team_df['opp_inhibitors']

In [6]:
# Visualizing data before imputation (if we decide to do it )
# (only the winning games, otherwise the data will always be symmetric)

# Visualize the distribution of gold difference columns
gold_diff_columns = ['golddiffat10', 'golddiffat15', 'golddiffat20']
for col in gold_diff_columns:
    fig = px.histogram(
        team_df[team_df['result'] == 1],
        x=col,
        nbins=20,
        title=f'Distribution of {col}',
        labels={col: 'Gold Difference'},
        template='plotly_white',
        color_discrete_sequence=['gold']
    )
    fig.update_layout(
        bargap=0.1,
        xaxis_title='Gold Difference',
        yaxis_title='Frequency',
        font=dict(size=14)
    )
    fig.show()

# Visualize the distribution of XP difference columns
xp_diff_columns = ['xpdiffat10', 'xpdiffat15', 'xpdiffat20']
for col in xp_diff_columns:
    fig = px.histogram(
        team_df[team_df['result'] == 1],
        x=col,
        nbins=20,
        title=f'Distribution of {col}',
        labels={col: 'XP Difference'},
        template='plotly_white',
        color_discrete_sequence=['blue']
    )
    fig.update_layout(
        bargap=0.1,
        xaxis_title='XP Difference',
        yaxis_title='Frequency',
        font=dict(size=14)
    )
    fig.show()

In [7]:
# Deciding against imputation here, as NaN values represent that a game has ended prior to that time
# and we want to keep that information

# Creating 3 features that we will plot against one another in the future
player_df['avg_xpdiff'] = player_df[xp_diff_columns].mean(axis=1, skipna=True)
player_df['avg_golddiff'] = player_df[gold_diff_columns].mean(axis=1, skipna=True)
player_df['game_length_min'] = player_df['gamelength'] / 60

In [8]:
fig = px.histogram(
    team_df,
    x='tower_advantage',
    nbins=15,  
    title='Distribution of Tower Advantage',
    labels={'tower_advantage': 'Tower Advantage (Team - Opponent)'},
    color_discrete_sequence=['indianred'] 
)

fig.update_layout(
    bargap=0.1, 
    template='plotly_white',  
    xaxis_title='Tower Advantage',
    yaxis_title='Number of Games',
    font=dict(size=14)
)

fig.show()

In [9]:
fig = px.histogram(
    team_df,
    x='inhibitor_advantage',
    nbins=15, 
    title='Distribution of Inhibitor Advantage',
    labels={'inhibitor_advantage': 'Inhibitor Advantage (Team - Opponent)'},
    color_discrete_sequence=['skyblue'] 
)

fig.update_layout(
    bargap=0.1, 
    template='plotly_white', 
    xaxis_title='Inhibitor Advantage',
    yaxis_title='Number of Games',
    font=dict(size=14)
)

fig.show()

In [10]:
# Creating a copy df
gold_stats = team_df.copy()[['gameid', 'earnedgold', 'side', 'result']]

gold_stats['red_win'] = ((gold_stats['result'] == 1) & ((gold_stats['side'] == 'Red')))
red_side_gold = gold_stats.loc[gold_stats['side'] == 'Red', ['gameid', 'earnedgold', 'red_win']]
blue_side_gold =  gold_stats.loc[gold_stats['side'] == 'Blue', ['gameid', 'earnedgold']]
gold_final = red_side_gold.merge(blue_side_gold, on='gameid', suffixes=('_red', '_blue'))

fig = px.scatter(
    gold_final,
    x='earnedgold_red',
    y='earnedgold_blue',
    color='red_win',
    color_discrete_map={True: '#D2042D', False: '#0047AB'},
    opacity=0.6,
    hover_data={
        'earnedgold_red': ':.0f',
        'earnedgold_blue': ':.0f',
        'red_win': True
    },
    labels={
        'earnedgold_red': 'Red Team Gold',
        'earnedgold_blue': 'Blue Team Gold',
        'red_win': 'Red Team Won'
    },
    title='Red Gold Earned vs Blue Gold Earned by Game Outcome'
)

fig.update_traces(marker=dict(size=6))

fig.update_layout(
    template='plotly_white', 
    xaxis=dict(title='Red Team Earned Gold', tickformat=',', title_font_size=16),
    yaxis=dict(title='Blue Team Earned Gold', tickformat=',', title_font_size=16),
    legend_title_text='Red Team Win?',
    legend=dict(font=dict(size=12))
)

fig.show()

In [11]:
# Here, we will sample 10% of the games
wins = player_df[player_df['result'] == 1]
all_games = wins['gameid'].drop_duplicates()
sampled_games = all_games.sample(frac=0.1, random_state=98)
sampled_df = wins[wins['gameid'].isin(sampled_games)]

In [12]:
fig = px.scatter(
    sampled_df,
    x='game_length_min',
    y='avg_xpdiff',
    color='position',
    title='Game Length (minutes) vs Average XP Difference by position (Winning Games)',
    labels={'avg_xpdiff': 'Average XP Difference', 'game_length_min': 'Game Length (minutes)'},
    template='plotly_white',
    hover_data=['position']
)

fig.update_layout(
    xaxis_title='Game Length (minutes)',
    yaxis_title='Average XP Difference',
    font=dict(size=14)
)

fig.show()

In [13]:
fig = px.scatter(
    sampled_df,
    x='game_length_min',
    y='avg_golddiff',
    color='position',
    title='Game Length (minutes) vs Average Gold Difference by position (Winning Games)',
    labels={'game_length_min': 'Game Length (minutes)', 'avg_golddiff': 'Average Gold Difference'},
    template='plotly_white',
    hover_data=['position']
)

fig.update_layout(
    xaxis_title='Game Length (minutes)',
    yaxis_title='Average Gold Difference',
    font=dict(size=14)
)

fig.show()

In [14]:
# Correlation of Gold and XP Difference with Win by Position
gold_corr = player_df.groupby('position').apply(lambda g: g['avg_golddiff'].corr(g['result']))
xp_corr = player_df.groupby('position').apply(lambda g: g['avg_xpdiff'].corr(g['result']))

imp = (pd
       .DataFrame({'gold': gold_corr, 'xp': xp_corr})
       .reset_index()
       .melt(id_vars='position', var_name='metric', value_name='corr'))

fig = px.bar(
    imp,
    x='position',
    y='corr',
    color='metric',
    barmode='group',
    title='Correlation of Gold/XP Difference with Win by Position',
    labels={'corr':'Correlation with Win','position':'Position','metric':'Metric'},
    template='plotly_white'
)
fig.update_layout(font=dict(size=14))
fig.show()

In [15]:
# Creating a pivot table for win rates by champion and position
# Example of how this reads: "Aatrox has a win rate of .535 in the top lane, not played in any other lane"
win_rates = player_df.groupby(['champion','position'])['result'] \
                    .agg(win_rate='mean', games='count') \
                    .reset_index()

pd.pivot_table(
    win_rates[win_rates['games'] >= 30],
    index='champion',
    columns='position',
    values='win_rate'
).sort_values(by='champion', ascending=True)

position,bot,jng,mid,sup,top
champion,,,,,
Aatrox,NaN,NaN,NaN,NaN,0.535484
Ahri,NaN,NaN,0.524441,NaN,NaN
Akali,NaN,NaN,0.513043,NaN,0.536748
Akshan,NaN,NaN,0.492063,NaN,0.507812
Alistar,NaN,NaN,NaN,0.495098,NaN
...,...,...,...,...,...
Zed,NaN,0.447368,0.580000,NaN,NaN
Zeri,0.525688,NaN,0.512821,NaN,NaN
Ziggs,0.421053,NaN,0.391304,NaN,NaN


## Step 3: Framing a Prediction Problem

<p> When a team is down by, say, >5k gold at 20 minutes, predict the probability they still win. </p>

## Step 4: Baseline Model

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [17]:
team_df = team_df[team_df['gamelength'] > (20 * 60)]

features = ['golddiffat10', 'xpdiffat10', 'csdiffat10', 'killsat10', 'assistsat10', 'deathsat10']
X = team_df[features]
y = team_df['result']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [18]:
svc_pipeline = make_pipeline(
    StandardScaler(),
    SVC(kernel='linear', random_state=42)
)

svc_pipeline.fit(X_train, y_train)
svc_preds = svc_pipeline.predict(X_test)

print(classification_report(y_test, svc_preds))

              precision    recall  f1-score   support

           0       0.69      0.69      0.69      2119
           1       0.69      0.69      0.69      2118

    accuracy                           0.69      4237
   macro avg       0.69      0.69      0.69      4237
weighted avg       0.69      0.69      0.69      4237



## Step 5: Final Model

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV

In [20]:
features = ['golddiffat20', 'xpdiffat20', 'csdiffat20', 'firstblood', 
            'firstherald', 'heralds', 'firstdragon', 'killsat20', 'assistsat20', 'deathsat20']

X = team_df[features]
y = team_df['result']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [21]:
class Advantage(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.feature_pairs = [
            ('dragons', 'opp_dragons'),
            ('barons', 'opp_barons'),
            ('towers', 'opp_towers'),
            ('inhibitors', 'opp_inhibitors')
        ] 

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_new = X.copy()
        for feat, opp_feat in self.feature_pairs:
            adv_name = f"{feat}_adv"
            X_new[adv_name] = X_new[feat] - X_new[opp_feat]
        return X_new

In [22]:
svc_imprv_pipeline = make_pipeline(
    StandardScaler(),
    SVC(random_state=42)
)


hyperparams = {
    'svc__C': [0.1, 1, 10, 100],               # Test different levels of regularization
    'svc__kernel': ['linear', 'rbf'],         # Try both simple and non-linear boundaries
    'svc__gamma': ['scale', 'auto', 0.01, 0.001],  # 'scale' and 'auto' are adaptive, 0.01/0.001 are manual
}

svc_model = GridSearchCV(
    estimator=svc_imprv_pipeline,
    param_grid=hyperparams,
    scoring='neg_mean_squared_error',
    n_jobs = -1
)

svc_model.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
svc_preds = svc_model.predict(X_test)
print(classification_report(y_test, svc_preds))